In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model

ModuleNotFoundError: No module named 'tensorflow'

# Load and Process Images

## Function to load and preprocess the image

In [4]:
def load_and_process_image(image_path):
    img = PIL.Image.open(image_path)
    img = img.resize((400, 400))  # Resize for consistency
    img = np.array(img, dtype=np.float32)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)  # Preprocess for VGG19
    return img


## Function to display images

In [6]:
def show_image(image, title="Image"):
    image = image[0]  # Remove batch dimension
    image = np.clip(image, 0, 255).astype('uint8')  # Clip to valid range
    plt.imshow(image)
    plt.axis('off')
    plt.title(title)
    plt.show()

# Define Content and Style Loss

## Function to compute content loss


In [9]:
def content_loss(base_content, target):
    return tf.reduce_mean(tf.square(base_content - target))

## Function to compute style loss using Gram matrix

In [11]:
def gram_matrix(tensor):
    channels = int(tensor.shape[-1])
    vectorized = tf.reshape(tensor, [-1, channels])
    gram = tf.matmul(tf.transpose(vectorized), vectorized)
    return gram / tf.cast(tf.size(vectorized), tf.float32)

## Compute style loss


In [13]:
def style_loss(base_style, gram_target):
    gram_base = gram_matrix(base_style)
    return tf.reduce_mean(tf.square(gram_base - gram_target))

# Load Pre-Trained VGG19 Model

In [15]:
# Load VGG19 model without the fully connected layers
def get_model():
    vgg = vgg19.VGG19(weights='imagenet', include_top=False)
    vgg.trainable = False  # Freeze the model
    
    # Layers used for content and style representation
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    
    outputs = {layer.name: vgg.get_layer(layer.name).output for layer in (style_layers + content_layers)}
    
    return Model(inputs=vgg.input, outputs=outputs), style_layers, content_layers


# Define Optimization and Training

In [17]:
# Extract features from model
def get_features(image, model, style_layers, content_layers):
    outputs = model(image)
    style_features = [outputs[layer] for layer in style_layers]
    content_features = [outputs[layer] for layer in content_layers]
    return style_features, content_features

# Training function
def train(content_path, style_path, epochs=500, alpha=1e4, beta=1e-2):
    # Load images
    content_image = load_and_process_image(content_path)
    style_image = load_and_process_image(style_path)

    # Load VGG model
    model, style_layers, content_layers = get_model()
    
    # Extract features
    style_features, _ = get_features(style_image, model, style_layers, content_layers)
    _, content_features = get_features(content_image, model, style_layers, content_layers)
    
    # Initialize generated image
    generated_image = tf.Variable(content_image, dtype=tf.float32)
    
    # Define optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=5.0)

    # Training loop
    for i in range(epochs):
        with tf.GradientTape() as tape:
            gen_style_features, gen_content_features = get_features(generated_image, model, style_layers, content_layers)

            # Compute total loss
            content_loss_value = content_loss(gen_content_features[0], content_features[0])
            style_loss_value = sum(style_loss(gen, target) for gen, target in zip(gen_style_features, style_features))
            total_loss = alpha * content_loss_value + beta * style_loss_value

        # Compute gradients and update generated image
        grad = tape.gradient(total_loss, generated_image)
        optimizer.apply_gradients([(grad, generated_image)])
        generated_image.assign(tf.clip_by_value(generated_image, -128, 127))

        # Print progress
        if i % 100 == 0:
            print(f"Iteration {i}/{epochs} - Loss: {total_loss.numpy()}")

    return generated_image


# Run the Style Transfer

In [19]:
# Provide paths to your images
content_path = 'tsuyoshi-kozu-VuSiUScxre0-unsplash.jpg'
style_path = 'pexels-kuremo-8660213.jpg'

# Run the training
stylized_image = train(content_path, style_path)

# Display the final output
show_image(stylized_image.numpy(), title="Stylized Image")


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


AttributeError: 'str' object has no attribute 'name'